In [ ]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import sys 
import DSI_Capstone_Steemit
from DSI_Capstone_Steemit.utils.utils import(
    load_data_and_description,
    load_raw_cleaned
)
import networkx as nx
# load_data_and_description(data_type = 'tfidf')
data_directory = '../data/'

%matplotlib inline

import seaborn as sns
import community
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import community_louvain as community
input_directory = os.path.join(data_directory,'networkx_votes')

def load_joblib(filename):
    return joblib.load(os.path.join(input_directory,filename))

In [ ]:
authors = pd.read_csv(os.path.join(data_directory,'accounts.csv'))

In [ ]:
raw_posts = load_raw_cleaned()



In [ ]:
raw_posts = raw_posts[raw_posts['depth'] == 0]

In [ ]:
hubs,authorities = load_joblib('hits')
cluster = load_joblib('parts')
pagerank = load_joblib('prank') 
eig_cent = load_joblib('eig_cent') 
core_k = load_joblib('core_k') 




In [ ]:


raw_posts['Cluster'] = raw_posts['author'].map(cluster)
raw_posts.loc[:,'Cluster Condense'] = raw_posts['Cluster']
raw_posts.loc[~raw_posts['Cluster'].isin([1,3,0,2,5,4]),'Cluster Condense'] = 'Other'
raw_posts['Hubs'] = raw_posts['author'].map(hubs) * 10000
raw_posts['Authorities'] = raw_posts['author'].map(authorities) * 10000
raw_posts['Page Rank'] = raw_posts['author'].map(pagerank) * 10000
raw_posts['Eigen Centrality'] = raw_posts['author'].map(eig_cent)* 10000
raw_posts['Core K'] = raw_posts['author'].map(core_k)*10000


authors['Hubs'] = authors['name'].map(hubs) * 10000
authors['Authorities'] = authors['name'].map(authorities) * 10000
authors['Cluster'] = authors['name'].map(cluster)
authors.loc[:,'Cluster Condense'] = authors['Cluster']
authors.loc[~authors['Cluster'].isin([1,3,0,2,5,4]),'Cluster Condense'] = 'Other'


authors['Page Rank'] = authors['name'].map(pagerank) * 10000
authors['Eigen Centrality'] = authors['name'].map(eig_cent)* 10000
authors['Core K'] = authors['name'].map(core_k)*10000

In [ ]:
posts_author_mean = raw_posts.groupby('author').mean()
posts_author_sum = raw_posts.groupby('author').sum()





posts_cluster_mean = raw_posts.groupby(['Cluster Condense']).mean().ix[:,['total_payout_value']].reset_index()
posts_cluster_mean.columns = ['Cluster','Mean Payout Value']
posts_cluster_sum = raw_posts.groupby(['Cluster Condense']).sum().ix[:,['total_payout_value']].reset_index()
posts_cluster_sum.columns = ['Cluster','Summed Payout Value']

posts_cluster_author_mean = authors.groupby(['Cluster Condense']).mean().ix[:,['reputation']].reset_index()
posts_cluster_author_mean.columns = ['Cluster','Average Author Reputation']

posts_cluster_author_sum = authors.groupby(['Cluster Condense']).mean().ix[:,['reputation']].reset_index()
posts_cluster_author_sum.columns = ['Cluster','Summed Author Reputation']



In [ ]:
fig,axs = plt.subplots(2,2,figsize = (8,20))

ax1 = plt.subplot2grid((7,4), (0,0), colspan=4)
ax2 = plt.subplot2grid((7,4), (1,0), colspan=2)
ax3 = plt.subplot2grid((7,4), (1, 2), colspan=2)
ax4 = plt.subplot2grid((7,4), (2, 0), colspan=2)
ax5 = plt.subplot2grid((7,4), (2, 2), colspan=2)
ax6 = plt.subplot2grid((7,4), (3, 0), colspan=2)
ax7 = plt.subplot2grid((7,4), (3, 2), colspan=2)
ax8 = plt.subplot2grid((7,4), (4, 0), colspan=2)
ax9 = plt.subplot2grid((7,4), (4, 2), colspan=2)
ax10 = plt.subplot2grid((7,4), (5, 0), colspan=2)
ax11 = plt.subplot2grid((7,4), (5, 2), colspan=2)




ax_it =iter([ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11])

alpha = 0.4
ax = ax_it.next()
order = np.sort(raw_posts['Cluster Condense'].unique())
sns.countplot(y = 'Cluster Condense',data=raw_posts,order=order,ax=ax)


ax = ax_it.next()


sns.barplot(y = 'Cluster',
            x = 'Summed Payout Value',
            ax = ax,
            data=posts_cluster_sum)
ax.set_xlabel('Summed Payout Value')


ax = ax_it.next()
sns.barplot(y = 'Cluster Condense',x = 'total_payout_value',ax = ax,order=order,
            data=raw_posts)
ax.set_xlabel('Mean Payout Value')



ax = ax_it.next()
sns.barplot(y = 'Cluster',
            x = 'Summed Author Reputation',order=order,
            ax = ax,
            data=posts_cluster_author_sum)
ax.set_xlabel('Summed Author Reputation')
ax = ax_it.next()
sns.barplot(y = 'Cluster Condense',
            x = 'reputation',
            ax = ax,order=order,
            data=authors)
ax.set_xlabel('Average Author Reputation')



ax = ax_it.next()
sns.barplot(y = 'Cluster Condense',
            x = 'Page Rank',
            ax = ax,order=order,
            data=authors)
ax.set_xlabel('Average Eigen Centrality')



ax = ax_it.next()
sns.barplot(y = 'Cluster Condense',
            x = 'Eigen Centrality',
            ax = ax,order=order,
            data=authors)
ax.set_xlabel('Average Page Rank')

ax = ax_it.next()
sns.barplot(y = 'Cluster Condense',
            x = 'Authorities',
            ax = ax,order=order,
            data=authors)
ax.set_xlabel('Authorities')

ax = ax_it.next()
sns.barplot(y = 'Cluster Condense',
            x = 'Hubs',
            ax = ax,order=order,
            data=authors)
ax.set_xlabel('Hubs')

ax = ax_it.next()
sns.barplot(y = 'Cluster Condense',
            x = 'Core K',
            ax = ax,order=order,
            data=authors)
ax.set_xlabel('Average Core K')



plt.tight_layout()

# Relationships between Measurements


In [ ]:
cols = ['Hubs','Authorities','Page Rank','Eigen Centrality','Core K','reputation']
sns.pairplot(authors,vars = cols,diag_kind="kde",plot_kws = {'alpha':0.1})

# Relationship between Measurements and Author Reputation

In [ ]:
fig,axs = plt.subplots(4,2,figsize = (20,20))
ax_it =iter(axs.flatten())

alpha = 0.4

for col in ['Hubs','Authorities','Page Rank','Eigen Centrality','Core K']:
    ax = ax_it.next()
    sns.regplot(data=authors,x = col,y = 'reputation',ax = ax,scatter_kws={"alpha": alpha})


# Relationship between Measurements and Post Value

In [ ]:
fig,axs = plt.subplots(4,2,figsize = (20,20))
ax_it =iter(axs.flatten())

alpha = 0.4

for col in ['Hubs','Authorities','Page Rank','Eigen Centrality','Core K','author_reputation_scaled']:
    ax = ax_it.next()
    sns.regplot(data=raw_posts,x = col,y = 'total_payout_value',ax = ax,scatter_kws={"alpha": alpha})


In [ ]:
raw_posts.groupby('author')

In [ ]:
fig,axs = plt.subplots(4,2,figsize = (20,20))
ax_it =iter(axs.flatten())

alpha = 0.4

for col in ['Hubs','Authorities','Page Rank','Eigen Centrality','Core K','author_reputation']:
    ax = ax_it.next()
    sns.regplot(data=posts_author_mean,x = col,y = 'total_payout_value',ax = ax,scatter_kws={"alpha": alpha})



In [ ]:
posts_author_mean_cluster

In [ ]:
fig,axs = plt.subplots(4,2,figsize = (20,20))
ax_it =iter(axs.flatten())
mcmap = plt.cm.get_cmap('Vega20')


alpha = 1

posts_author_mean_cluster = raw_posts.groupby(['Cluster Condense']).mean().reset_index()


def regplot_hue(data,x,y,hue,ax):
    for i,hue_value in enumerate(np.sort(data[hue].unique())):
        color = mcmap(i)
        idx = data[hue] == hue_value
        df_plot = data[idx]
        sns.regplot(data=df_plot,
                    x = x,
                    y = y,
                    ax = ax,
                    scatter_kws={"alpha": alpha},fit_reg = False,
                    label=str(hue_value), color = color)


for col in ['Hubs','Authorities','Page Rank','Eigen Centrality','Core K','author_reputation']:
    ax = ax_it.next()
    regplot_hue(data=posts_author_mean_cluster,x = col,y = 'total_payout_value',ax = ax,hue = 'Cluster Condense')
    ax.legend()


In [ ]:
fig,axs = plt.subplots(4,2,figsize = (20,20))
ax_it =iter(axs.flatten())
mcmap = plt.cm.get_cmap('Vega20')


alpha = 0.4

posts_author_mean_cluster = raw_posts.groupby(['Cluster Condense','author']).mean().reset_index()


def regplot_hue(data,x,y,hue,ax):
    for i,hue_value in enumerate(np.sort(data[hue].unique())):
        color = mcmap(i)
        idx = data[hue] == hue_value
        df_plot = data[idx]
        sns.regplot(data=df_plot,
                    x = x,
                    y = y,
                    ax = ax,
                    scatter_kws={"alpha": alpha},fit_reg = False,
                    label=str(hue_value), color = color)


for col in ['Hubs','Authorities','Page Rank','Eigen Centrality','Core K','author_reputation']:
    ax = ax_it.next()
    regplot_hue(data=posts_author_mean_cluster,x = col,y = 'total_payout_value',ax = ax,hue = 'Cluster Condense')
    ax.legend(loc = 'best')
    


In [ ]:
posts_author_mean.head()

## Cluster Exploration

In [ ]:
# fig,axs = plt.subplots(3,2,figsize = (20,20))
# ax_it =iter(axs.flatten())

# def regplot_hue(data,x,y,hue,ax):
#     for i,hue_value in enumerate(np.sort(data[hue].unique())):
#         color = mcmap(i)
#         idx = data[hue] == hue_value
#         df_plot = data[idx]
#         sns.regplot(data=df_plot,x = x,y = y,ax = ax,scatter_kws={"alpha": alpha},label=str(i), color = color)


# alpha = 0.4
# ax = ax_it.next()
# # sns.regplot(data=authors,x = 'Hubs',y = 'Authorities',ax = ax,scatter_kws={"alpha": alpha})
# regplot_hue(data=authors,x = 'Hubs',y = 'Authorities',ax = ax,hue = 'Cluster Condense')
# ax.legend(loc= 'best')

# ax = ax_it.next()
# regplot_hue(data=authors,x = 'Page Rank',y = 'Eigen Centrality',ax = ax,hue = 'Cluster Condense')
# ax.legend(loc= 'best')

# ax = ax_it.next()
# regplot_hue(data=authors,x = 'Hubs',y = 'Page Rank',ax = ax,hue = 'Cluster Condense')
# ax.legend(loc= 'best')

# ax = ax_it.next()
# regplot_hue(data=authors,x = 'Hubs',y = 'Eigen Centrality',ax = ax,hue = 'Cluster Condense')
# ax.legend(loc= 'best')

# ax = ax_it.next()
# regplot_hue(data=authors,x = 'Authorities',y = 'Page Rank',ax = ax,hue = 'Cluster Condense')

# ax.legend(loc= 'best')


# ax = ax_it.next()
# regplot_hue(data=authors,x = 'Authorities',y = 'Eigen Centrality',ax = ax,hue = 'Cluster Condense')
# ax.legend(loc= 'best')


